#### Agora iremos estudar as métricas

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

In [6]:
data = pd.read_csv('creditcard.csv')[:80000]

data.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


<small>Casos de Fraude possuem Classe 1, já sem Fraude possuem Classe 0</small>

In [7]:
X = data.drop(columns=['Time', 'Amount', 'Class']).values
y = data['Class'].values
f'Shapes of X={X.shape} y={y.shape}, #Fraud Cases={y.sum()}'

'Shapes of X=(80000, 28) y=(80000,), #Fraud Cases=196'

É bom levar em conta que o número de casos de fraude é muito baixo quando comparamos com o total de amostras, o que pode deixar o modelo tendencioso.

In [11]:
from sklearn.linear_model import LogisticRegression

# Para tentar lidar com o problema, vamos atribuir peso as classes
# Como a classe 1 está em minoria, iremos penalizar mais erros relacionados a está classe
model = LogisticRegression(class_weight={0:1, 1:2}, max_iter=1000)
model.fit(X,y).predict(X).sum()

np.int64(172)

Iremos realizar o GridSearchCV para achar o melhor valor para class_weight, afim de priorizar mais a classe Fraude

In [12]:
from sklearn.model_selection import GridSearchCV

In [18]:
from sklearn.metrics import precision_score, recall_score, make_scorer

precision_score(y, grid.predict(X))

0.7814569536423841

In [27]:
grid = GridSearchCV (
  estimator= LogisticRegression(max_iter=1000),
  # Aqui estamos usando um laço para percorrer os possíveis valores de v
  param_grid={'class_weight': [{0:1, 1:v} for v in np.linspace(1,20,30)]},
  # Passa as duas medidas de precisão
  scoring={'precision': make_scorer(precision_score), 'recall_score': make_scorer(recall_score)},
  # Escolhe por meio do refit qual das duas medidas priorizar
  refit='precision',
  return_train_score=True,
  cv = 10
)

In [28]:
grid.fit(X,y)

c:\Users\PICHAU\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


GridSearchCV(cv=10, estimator=LogisticRegression(max_iter=1000),
             param_grid={'class_weight': [{0: 1, 1: np.float64(1.0)},
                                          {0: 1,
                                           1: np.float64(1.6551724137931034)},
                                          {0: 1,
                                           1: np.float64(2.310344827586207)},
                                          {0: 1,
                                           1: np.float64(2.9655172413793105)},
                                          {0: 1,
                                           1: np.float64(3.6206896551724137)},
                                          {0: 1,
                                           1: np.float64(4.275862068965517)},
                                          {0: 1,
                                           1: np.float64(4.931034482758621)},
                                          {0: 1...
                                           1: np.float64(17.379310344827587)},
                                          {0: 1,
                                           1: np.float64(18.03448275862069)},
                                          {0: 1,
                                           1: np.float64(18.689655172413794)},
                                          {0: 1,
                                           1: np.float64(19.344827586206897)},
                                          {0: 1, 1: np.float64(20.0)}]},
             refit='precision', return_train_score=True,
             scoring={'precision': make_scorer(precision_score, response_method='predict'),
                      'recall_score': make_scorer(recall_score, response_method='predict')})

Precision_score mede a proporção de previsões positivas corretas em relação ao total de previsões positivas feitas pelo modelo.<br>
Ou seja, neste caso mede a razão entre fraudes acertadas sobre o total de previsões de fraude feitas, fraudes_certas / (fraudes_certas + fraudes_erradas).

Já recall_score, mede a proporção de previsões positivas corretas em relação ao número de exemplos reais, fraudes_corretas / (total_fraudes_reais)

In [31]:
pd.DataFrame(grid.cv_results_).head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,params,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,...,split2_train_recall_score,split3_train_recall_score,split4_train_recall_score,split5_train_recall_score,split6_train_recall_score,split7_train_recall_score,split8_train_recall_score,split9_train_recall_score,mean_train_recall_score,std_train_recall_score
0,0.125020,0.021122,0.004039,0.000594,"{0: 1, 1: 1.0}","{'class_weight': {0: 1, 1: 1.0}}",1.0,0.463415,0.583333,1.0,...,0.632768,0.559322,0.573864,0.573864,0.562500,0.619318,0.625000,0.602273,0.611049,0.051750
1,0.119145,0.015638,0.003875,0.000373,"{0: 1, 1: 1.6551724137931034}","{'class_weight': {0: 1, 1: 1.6551724137931034}}",1.0,0.441860,0.583333,1.0,...,0.689266,0.627119,0.670455,0.647727,0.630682,0.681818,0.698864,0.687500,0.681366,0.053324
2,0.117025,0.008520,0.004063,0.000511,"{0: 1, 1: 2.310344827586207}","{'class_weight': {0: 1, 1: 2.310344827586207}}",1.0,0.431818,0.583333,1.0,...,0.734463,0.683616,0.715909,0.693182,0.681818,0.721591,0.744318,0.727273,0.723889,0.044372
3,0.118089,0.010282,0.003847,0.000368,"{0: 1, 1: 2.9655172413793105}","{'class_weight': {0: 1, 1: 2.9655172413793105}}",1.0,0.431818,0.583333,1.0,...,0.779661,0.706215,0.744318,0.732955,0.715909,0.755682,0.772727,0.738636,0.749413,0.039118
4,0.118902,0.016659,0.003802,0.000281,"{0: 1, 1: 3.6206896551724137}","{'class_weight': {0: 1, 1: 3.6206896551724137}}",1.0,0.431818,0.583333,1.0,...,0.819209,0.734463,0.761364,0.744318,0.732955,0.778409,0.784091,0.761364,0.770939,0.035655
